In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import summary_table 
from scipy.optimize import minimize

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
from WindPy import w
w.start() # 默认命令超时时间为120秒，如需设置超时时间可以加入waitTime参数，例如waitTime=60,即设置命令超时时间为60秒  
w.isconnected() # 判断WindPy是否已经登录成功

In [29]:
#从万得获取价格数据并计算收益率
def get_price(fund_code,benchmark_code,stock_index_code,debt_index_code,start_date,end_date,PriceAdj):
    fund = w.wsd(fund_code, "close", start_date, end_date, PriceAdj)             #usedf=True
    benchmark = w.wsd(benchmark_code, "close", start_date, end_date, PriceAdj)
    stock_index = w.wsd(stock_index_code, "close", start_date, end_date, PriceAdj)
    debt_index = w.wsd(debt_index_code ,"close", start_date, end_date, PriceAdj)
    raw_price = pd.DataFrame({fund_code[0]:fund.Data[0],
                         benchmark_code[0]:benchmark.Data[0],
                         stock_index_code[0]:stock_index.Data[0],
                         stock_index_code[1]:stock_index.Data[1],
                         stock_index_code[2]:stock_index.Data[2],
                         stock_index_code[3]:stock_index.Data[3],
                         stock_index_code[4]:stock_index.Data[4],
                         stock_index_code[5]:stock_index.Data[5],
                         debt_index_code[0]: debt_index.Data[0]},index=fund.Times)
   
    return raw_price

def get_price_return(raw_price):
    price_return = pd.DataFrame(np.array(raw_price.iloc[:-1,:])/np.array(raw_price.iloc[1:,:])-1,
                            columns= fund_code+benchmark_code+stock_index_code+debt_index_code, index=raw_price.index[1:])
    return price_return

In [49]:
#规划求解系数
def solve_slsqp_params(start_date,end_date,period,PriceAdj,fund_code,benchmark_code,stock_index_code,debt_index_code):
    #获取数据并计算收益率
    raw_price = get_price(fund_code,benchmark_code,stock_index_code,debt_index_code,start_date,end_date,PriceAdj)
    price_return = get_price_return(raw_price)
    #时间分段
    time_list = w.tdays(start_date, end_date, period,"Days=Alldays")
    #定义数据框记录规划求解的结果
    slsqp_params = pd.DataFrame(index=time_list.Times,
                            columns=stock_index_code+debt_index_code
                                     +['R_squared','1-R_squared']
                                     +['fund_return','style_return','benchmark_return'])
    #最小化目标函数：
    def fun(args):
        x0,x1,x2,x3,x4,x5,x6,y = args
        def sq2(a):
            s = 0
            for j in range(len(y)):
                s += (y[j]-a[0]*x0[j]-a[1]*x1[j]-a[2]*x2[j]-a[3]*x3[j]-a[4]*x4[j]-a[5]*x5[j]-a[6]*x6[j])**2
            return s
        return sq2 
    #风格系数约束条件:
    def cons(args):
        stock_low = args
        con = ({'type': 'eq', 'fun': lambda a:  sum(a)-1},\
               {'type': 'ineq', 'fun': lambda a: sum(a[:-1])-stock_low})
        return con
    #循环各期求解:
    for i in range(len(time_list.Times)):
        #按照区间将价格序列和收益率序列分段
        if i == 0:
            raw_price_interval = raw_price[raw_price.index<=time_list.Times[i]]
            price_return_interval = price_return[price_return.index<=time_list.Times[i]]
        else:
            raw_price_interval = raw_price[(raw_price.index<=time_list.Times[i]) & (raw_price.index>=time_list.Times[i-1])]
            price_return_interval = price_return[(price_return.index<=time_list.Times[i]) & (price_return.index>time_list.Times[i-1])]
        #提取回归变量序列
        x = price_return_interval[stock_index_code+debt_index_code]
        y = price_return_interval[fund_code]
        for k in range(len(stock_index_code+debt_index_code)):
            vars()['x'+str(k)] = np.array(x.iloc[:,k])
        y = np.array(y)
        #SLSQP算法规划求解：最小化残差平方和
        stock_low = 0.8
        args_1 = (x0,x1,x2,x3,x4,x5,x6,y)
        args_2 = stock_low
        bounds = ((0,1), (0,1), (0,1), (0,1), (0,1), (0,1),(0,1))   #各个风格系数的取值范围
        a0 = np.array([0,0,0,0,0,0,0])  #初始化系数的值，注意不同初始化值会影响最后系数的取值大小，但最小目标值差异很小
        vars()['res_'+str(i)] = minimize(fun(args_1), a0, method='SLSQP',constraints=cons(args_2), bounds=bounds)
        #计算R方
        sse = vars()['res_'+str(i)].fun
        sst = 0
        for j in range(len(y)):
            sst += (y[j]-y.mean())**2
        R2 = 1-sse/sst
        #风格回归系数
        slsqp_params.loc[time_list.Times[i],stock_index_code+debt_index_code] = vars()['res_'+str(i)].x
        #R方
        slsqp_params.loc[time_list.Times[i],['R_squared','1-R_squared']] = [R2[0],1-R2[0]]
        #基金区间收益
        slsqp_params.loc[time_list.Times[i],fund_return] = raw_price_interval.iloc[-1][fund_code]/raw_price_interval.iloc[0][fund_code]-1
        #风格区间收益
        slsqp_params.loc[time_list.Times[i],style_return] = np.inner(np.array(raw_price_interval.iloc[-1][stock_index_code+debt_index_code])/np.array(raw_price_interval.iloc[0][stock_index_code+debt_index_code])-1,np.array(vars()['res_'+str(i)].x))
        #基准区间收益
        slsqp_params.loc[time_list.Times[i],benchmark_return] = raw_price_interval.iloc[-1][benchmark_code]/raw_price_interval.iloc[0][benchmark_code]-1
    return slsqp_params

In [50]:
#基金及指数代码列表

jc_index_code_list = ["399372.SZ","399373.SZ","399374.SZ","399375.SZ","399376.SZ","399377.SZ"]

sw_index_code_list = ["801811.SI","801812.SI","801813.SI","801821.SI","801822.SI","801823.SI","801831.SI","801832.SI","801833.SI",
                 "801841.SI","801842.SI","801843.SI","801851.SI","801852.SI","801853.SI","801863.SI"]

wind_index_code_list = ["8888701.WI","8888702.WI","8888703.WI","8888704.WI","8888705.WI","8888706.WI",
                        "8888707.WI","8888708.WI","8888709.WI","8888710.WI"]

zxbp_index_code_list = ["816000G.CI","816000PG.CI","816000PV.CI","816000V.CI","CSPSADRP.CI"]

debt_index_code = ['H11001.CSI']

fund_code = ["161005.OF"]

benchmark_code =["000300.SH"]

In [51]:
#时间区间及频率设置
PriceAdj = "PriceAdj = F" #前复权
start_date = "2015-01-01"
end_date = "2020-12-31"
period = "Period=S"  #半年度
stock_index_code = jc_index_code_list

In [52]:
slsqp_params = solve_slsqp_params(start_date,end_date,period,PriceAdj,fund_code,benchmark_code,stock_index_code,debt_index_code)

NameError: name 'x0' is not defined